## 예제 2.1 토큰화 코드
# 띄어쓰기를 기준으로

In [1]:
# 띄어쓰기 단위로 분리
input_text = "나는 오늘 커피를 먹었다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '오늘', '커피를', '먹었다']
str2idx:  {'나는': 0, '오늘': 1, '커피를': 2, '먹었다': 3}
idx2str:  {0: '나는', 1: '오늘', 2: '커피를', 3: '먹었다'}
input_ids:  [0, 1, 2, 3]


## 예제 2.2 토큰 아이디에서 벡터로 변환
# 토큰을 임베딩으로 변환(토큰 임베딩)


In [3]:
import torch
import torch.nn as nn

# 차원설정
embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (4, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 4, 16)
input_embeddings.shape
# 한개의 문장이 4개의 토큰을 가지고 16차원의 임베딩이 생성
# 입력 토큰 아이디를 16차원의 임의의 숫자 집합으로 변환, 의미는 부여 X

torch.Size([1, 4, 16])

## 예제 2.3 절대적 위치 인코딩
### 입력 토큰의 위치에 따라 고정된 임베딩을 더해준다.

In [4]:
embedding_dim = 16
max_position = 12
# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids)) # (4, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 4, 16)
# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 4, 16])

## 예제 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층
embedding_dim 16차원의 입력벡터를 head_dim 16차원으로 바꾸기

- query는 찾고 싶은 정보
- key는 각 단어 혹은 문서가 가진 특징
- values는 토큰의 진짜 내용, 문서를 제공할때 문서 값

In [10]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)
# 변환 수행
querys = weight_q(input_embeddings) # (1, 4, 16)
keys = weight_k(input_embeddings) # (1, 4, 16)
values = weight_v(input_embeddings) # (1, 4, 16)

querys.shape


torch.Size([1, 4, 16])

## 예제 2.5. 스케일 점곱 방식의 어텐션
1. 쿼리와 키 값을 곱
2. 분산이 커지는 것을 방지하기 위해 임베딩 차원의 수(dim_k)의 제곱근으로 나눈다.
3. 스코어가 1이 되게 하기위해 소프트맥스를 취해 가중치로 바꾼다
4. 가중치와 값을 곱해 입력과 동일한 형태의 출력을 반환

In [9]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16차원수 구하기
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)    # @ 행렬곱 쿼리 곱하기 키 값 나누기 차원의 수의 제곱근
	weights = F.softmax(scores, dim=-1)          # 어떤 토큰에 얼만큼 집중할지 확률값(가중치) 만들기
	return weights @ values

## 예제 2.6. 어텐션 연산의 입력과 출력

### 입력과 동일한 형태의 출력

In [11]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)
# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 4, 16])
어텐션 적용 후 형태:  torch.Size([1, 4, 16])


## 예제 2.7. 어텐션 연산을 수행하는 AttentionHead 클래스
### 지금까지 한거 한번에

In [12]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

## 예제 2.8. 멀티 헤드 어텐션 구현
###여러 어텐션의 적용해 성능을 높이자
### 토큰 사이의 관계를 여러 측면에서 동시에 고려(n_head 만큼 어텐션 연산 수행)
### 여러개의 문장이 있는게 아니라 한 문장에 다양한 관점으로 바로보기 여러개의 관점만큼 가중치가 생김

In [13]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    attention = compute_attention(querys, keys, values, self.is_causal)
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 4, 16])

## 예제 2.9. 층 정규화 코드
### 토큰별 벡터를 정규화 토큰의 개수와 상관없이 진행

In [14]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[ 2.9802e-08,  1.4901e-08, -3.7253e-08,  0.0000e+00]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328]]))

## 예제 2.10. 피드 포워드 층 코드
### 멀티 헤드 어텐션은 단어 사이의 관계를 파악
### 피드 포워드 층은 입력 텍스트 전체를 이해
### 선형층, 드롭아웃층, 층정규화, 활성함수로 구성


- Self-Attention의 출력(문맥 정보 포함된 벡터)이 →
FeedForward 네트워크의 입력으로 들어가서
각 토큰을 더 정교하고 의미 있게 표현

In [15]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1          d_model > dim_feedforward 선형변환
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2          dim_feedward > d_model 원래대로 다시 선형변환
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)    # 사전 정규화
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

## 예제 2.11. 인코더 층

In [16]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x

## 예제 2.12. 인코더 구현

In [17]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
        output = mod(output)
    return output

## 예제 2.13. 디코더에서 어텐션 연산(마스크 어텐션)

In [18]:
def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)
	if is_causal:
		query_length = querys.size(2)
		key_length = keys.size(2)
		temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)            # torch.ones 로 nxn행렬 만들고 .trill로 lower triangular를 만든다 >> 이게 마스크
		scores = scores.masked_fill(temp_mask == False, float("-inf"))                                 # 토큰 1은 1행의 [1,0,0,0]만 고려 토큰 2는 2행의 [1,1,0,0]만 고려 이런식으로 미래의 토큰값 고려못하게 함
	weights = F.softmax(scores, dim=-1) # (1, 5, 5)
	return weights @ values # (1, 5, 16)

## 예제 2.14. 크로스 어텐션이 포함된 디코더 층
### 크로스 어텐션 : 디코더에서 입력 문장의 정보를 참고하는 어텐션

In [19]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x

## 예제 2.15. 디코더 구현

In [20]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
        output = mod(output, src)
    return output